In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2

## Erstelle Augmented Bilder

In [ ]:
# Define the path to the dataset folder
data_folder = "Bilder"

# Define the new subfolder name for augmented images
new_subfolder = "Augmented"

# Define the path to the dataset folder
data_folder = "Bilder"

# Define the new subfolder name for augmented images
new_subfolder = "Augmented"

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Create a new subfolder for augmented images
        new_folder_path = os.path.join(folder_path, new_subfolder)
        os.makedirs(new_folder_path, exist_ok=True)
        # Iterate over the files in the folder
        for file_name in os.listdir(folder_path):
            # Assuming the files are image files, you can adjust the file type according to your dataset
            # Read the image using OpenCV
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Perform image augmentation
                    # Resize the image to a desired size
                    resized_image = cv2.resize(image, (300, 300))
                    datagen = ImageDataGenerator(
                        rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        fill_mode='nearest'
                    )
                    # Reshape the image to add batch dimension
                    reshaped_image = resized_image.reshape((1,) + resized_image.shape)
                    # Generate and save augmented images
                    i = 0
                    for batch in datagen.flow(reshaped_image, batch_size=1, save_to_dir=new_folder_path, save_prefix='aug', save_format='png'):
                        i += 1
                        if i >= 100:
                            break  # Break the loop after generating 100 augmented images
                else:
                    print(f"Failed to load image: {image_path}")
            else:
                print(f"Skipping non-image file: {file_name}")
           

            

## Vektorize Bilder und erstelle Labels dazu

In [ ]:
# Initialize lists to store images and labels
images = []
labels = []

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Get the label from the folder name
        label = folder_name
        # Get the path to the augmented folder
        augmented_folder_path = os.path.join(folder_path, new_subfolder)
        # Check if the augmented folder exists
        if os.path.exists(augmented_folder_path):
            # Iterate over the files in the augmented folder
            for file_name in os.listdir(augmented_folder_path):
                # Read the image using OpenCV
                image_path = os.path.join(augmented_folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Add the image to the list of images
                    images.append(image)
                    # Add the label to the list of labels
                    labels.append(label)
                else:
                    print(f"Failed to load image: {image_path}")

print(f"Total images: {len(images)}")
print(f"Total labels: {len(labels)}")
print(images[300].shape)
print(labels[300])


## Hog Feature Extraction

In [ ]:
from skimage.feature import hog
import cv2

# Initialize empty lists to store HOG features and labels
hog_features = []
hog_labels = []

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Get the path to the augmented folder
        augmented_folder_path = os.path.join(folder_path, new_subfolder)
        # Check if the augmented folder exists
        if os.path.exists(augmented_folder_path):
            # Iterate over the files in the augmented folder
            for file_name in os.listdir(augmented_folder_path):
                # Read the image using OpenCV
                image_path = os.path.join(augmented_folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Convert the image to grayscale
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    # Perform HOG feature extraction
                    hog_feature = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
                    # Add the HOG feature to the list of features
                    hog_features.append(hog_feature)
                    # Add the label to the list of labels
                    hog_labels.append(folder_name)
                    # Print image filename, HOG features, and label
                    print(f"Image: {file_name}, HOG features: {hog_feature}, Label: {folder_name}")
                else:
                    print(f"Failed to load image: {image_path}")

print(f"Total HOG features: {len(hog_features)}")
print(f"Total labels: {len(labels)}")


## LBP Feature Extraction

In [ ]:
from skimage.feature import local_binary_pattern

def extract_lbp_features(image, num_points, radius):
    lbp_features = []
    for channel in range(image.shape[2]):  # Iteriere über die Farbkanäle (RGB)
        lbp = local_binary_pattern(image[:, :, channel], num_points, radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        lbp_features.extend(hist)
    return lbp_features
 
# Initialize empty list to store LBP features
lbp_features = []
lbp_labels = []
 
# Parameter für LBP festlegen
num_points = 32
radius = 8


for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Get the path to the augmented folder
        augmented_folder_path = os.path.join(folder_path, new_subfolder)
        # Check if the augmented folder exists
        if os.path.exists(augmented_folder_path):
            # Iterate over the files in the augmented folder
            for file_name in os.listdir(augmented_folder_path):
                # Read the image using OpenCV
                image_path = os.path.join(augmented_folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    lbp_feature = extract_lbp_features(image, num_points, radius)
                    lbp_features.append(lbp_feature)
                    # Add the label to the list of labels
                    lbp_labels.append(folder_name)
                    print(f"Image: {file_name}, LBP features: {lbp_feature}, Label: {folder_name}")
                else:
                    print(f"Failed to load image: {image_path}")


In [ ]:
import matplotlib.pyplot as plt

# Initialize dictionaries to store average HOG features for each label
average_hog_features_by_label = {}

# Iterate over the labels
for label in set(labels):
    # Filter HOG features corresponding to the current label
    hog_features_for_label = [hog_feature for hog_feature, l in zip(hog_features, labels) if l == label]
    # Calculate the average HOG feature for the current label
    average_hog_feature = np.mean(hog_features_for_label, axis=0)
    # Store the average HOG feature for the current label
    average_hog_features_by_label[label] = average_hog_feature

# Plot average HOG features for each label
for label, avg_feature in average_hog_features_by_label.items():
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(avg_feature)), avg_feature, marker='o', linestyle='-', label=label)
    plt.title(f'Average HOG Feature Vector Visualization for Label: {label}')
    plt.xlabel('Feature Index')
    plt.ylabel('Average Feature Value')
    plt.legend()
    plt.grid(True)
    plt.show()

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
 
X_train, X_test, y_train, y_test = train_test_split(lbp_features, labels, test_size=0.2, random_state=42)
 
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
 
rf_classifier.fit(X_train, y_train)
 
accuracy = rf_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)
 
 
 
# Annahme: lbp_features ist eine Liste von LBP-Merkmalen und labels ist eine entsprechende Liste von Labels
 
# Aufteilung der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(lbp_features, labels, test_size=0.2, random_state=42)
 
# Initialisierung des Random-Forest-Klassifikators
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
 
# Trainieren des Random-Forest-Modells
rf_classifier.fit(X_train, y_train)
 
# Evaluierung des Modells (optional)
accuracy = rf_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)
 

Accuracy: 1.0
Accuracy: 1.0
